In [1]:
# imports
from downsample import *
import pandas as pd
from dask import delayed
import dask.dataframe as dd
import numpy as np

In [2]:
import dask
from dask.diagnostics import Profiler, ResourceProfiler, ProgressBar

In [6]:
#paths
datapath='../../datasets/'
apath = os.path.join(datapath, 'sample_tracks.csv')
bpath = os.path.join(datapath, 'sample_songs.csv')

In [13]:
# load datasets
# A = dd.to_delayed(dd.read_csv(apath, low_memory=False))
# B = dd.to_delayed(dd.read_csv(bpath, low_memory=False))
A = pd.read_csv(apath, low_memory=False)
B = pd.read_csv(bpath, low_memory=False)

In [5]:
spath = os.path.join(datapath, 'stopwords')
stopwords = get_stop_words(spath)
stopwords.extend(['and', 'in', 'the', 'my', 'me', 'to', 'you', 'i', 'andre', 'from', 'a', 'of', 'the', 'version', 'love', 'live', 'la', 'mix', 'album', 'dont'])
stopwords = list(set(stopwords))

In [7]:
def sample(df, size):
    return df.sample(size, replace=False)
def splitdf(df, nchunks):
    sample_splitted = np.array_split(df, nchunks)
    return sample_splitted

In [15]:
import numpy as np
def downsample_delayed(A, B, lid, rid, size, yparam, nlchunks=1, nrchunks=1, stopwords=[], compute=False):
    
    
    lsplitted = (delayed)(splitdf)(A, nlchunks)

    ltokens = []
    for i in xrange(nlchunks):
        lstrings = delayed(preprocess_table)(A, lid)
        tokens = delayed(tokenize_strings)(lstrings, stopwords)
        ltokens.append(tokens)
        
    invindex = delayed(build_inv_index)(ltokens)
    
    rsample = delayed(sample)(B, size)
    rsample_splitted = delayed(splitdf)(rsample, nrchunks)
    probe_results = []
    
    for i in xrange(nrchunks):
        rstrings = delayed(preprocess_table)(rsample_splitted[i], rid)
        rtokens = delayed(tokenize_strings)(rstrings, stopwords)
        probe_result = delayed(probe)(rtokens, invindex, yparam)
        probe_results.append(probe_result)
    
    result = delayed(postprocess)(probe_results, A, B, lid, rid)
    
    if compute:
            with ProgressBar():
                tmp = result.compute()
            return tmp   
    else:
        return result

In [21]:
res = downsample_delayed(A, B, 'id', 'id', 10000, 1, nlchunks=4, nrchunks=4, stopwords=stopwords, compute=True)

[########################################] | 100% Completed | 21.7s


In [19]:
res[0].head()

,id,title,year,episode,song,artists
3,3,Colburn Concert Series,2012.0,Colburn Orchestra Live Performance Conducted b...,From me flows what you call Time,tôru takemitsu
7,7,Kill the Irishman,2011.0,NaN,DAFFY DOTTY DAY,johnny caruso+jeffrey silverman amd roberta ...
14,14,Our Gang Follies of 1936,1935.0,NaN,How Ya Gonna Keep Em Down On The Farm?,walter donaldson+sam lewis+joe young+the bryan...
22,22,Heartbeat,1992.0,Outsiders (#1.8),Apache,the shadows+jerry lorden
35,35,Nirusu no fushigi na tabi,1980.0,NaN,Nils Holgersson,vince tempera+carlo+enzo e pino romano
